## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

- Felipe Cadavez Oliveira
  - 11208558
- Giovanni Shibaki Camargo
  - 11796444
- João Victor Sene Araujo
  - 11796382
- Lucas Keiti Anbo Mihara
  - 11796472

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);
2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \mbox{ se } j = i_s \\ 0 & \mbox{ caso contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = Pb
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

In [64]:
# Bibliotecas Usadas
import numpy as np
import networkx as nx
import random
import time
import scipy
import scipy.linalg
import scipy.sparse.linalg
import matplotlib.pyplot as plt

In [65]:
# Leitura dos arquivos manh.el e manh.xy
# E criação do grafo

# Leitura do arquivo manh.el para a criação da matriz de adjacencia
file1 = open('manh.el', 'r')
Lines = file1.readlines()

# Criação do grafo direcionado
Graph = nx.Graph();

max = 0
for line in Lines:
    values = line.split()
    x, y = values[0], values[1]
    if(int(x) > max):
        max = int(x)
    if(int(y) > max):
        max = int(y)

numNodes = max + 1
print("O grafo possui ",numNodes,"vértices")

for i in range(numNodes):
    Graph.add_node(i)

for line in Lines:
    values = line.split()
    x, y = values[0], values[1]
    Graph.add_edge(int(x), int(y))
    Graph.add_edge(int(y), int(x))

#Graph2 = Graph.subgraph(range(50))
# Plotando o grafo correspondente
#plt.figure(1, figsize=(200,200))
#nx.draw(Graph2, pos=nx.circular_layout(Graph2), with_labels=True)

O grafo possui  8837 vértices


In [66]:
# Leitura do arquivo manh.XY para as coordenadas dos pontos
count = 0
file2 = open('manh.xy', 'r')
Lines2 = file2.readlines()

for line in Lines2:
    values = line.split()
    x, y = values[0], values[1]

    count += 1

G2 = G.subgraph(max(nx.strongly_connected_components(G), key=len))

In [67]:
stronglyConnectedComponents = [Graph.subgraph(c).copy() for c in nx.connected_components(Graph)]

for item in stronglyConnectedComponents:
    print(item)
    print(nx.is_connected(item))

maximum = 0
for item in stronglyConnectedComponents:
    if item.number_of_nodes() > maximum:
        Graph2 = item
        maximum = item.number_of_nodes()

print("\n",Graph2)

Graph with 8708 nodes and 13556 edges
True
Graph with 123 nodes and 154 edges
True
Graph with 2 nodes and 1 edges
True
Graph with 2 nodes and 1 edges
True
Graph with 2 nodes and 1 edges
True

 Graph with 8708 nodes and 13556 edges


## Matriz de Laplace a partir matriz de Adjacencia e a matriz diagonal com os graus de cada vértice

In [68]:
# Matriz de laplace

laplaceMatrix = nx.adjacency_matrix(Graph2)

# Matriz de adjacência
print("Matriz de adjacencia")
MA = nx.to_numpy_array(Graph2)
print(MA)

print()

# Matriz diagonal com os graus de cada vértice
MD = np.diag(np.sum(MA,axis=1))

print()
print("Matriz diagonal com os graus dos vértices")
print(MD)
print()

C:\Users\giova\AppData\Local\Temp\ipykernel_13380\3566922698.py:3: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  laplaceMatrix = nx.adjacency_matrix(Graph2)


Matriz de adjacencia
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Matriz diagonal com os graus dos vértices
[[3. 0. 0. ... 0. 0. 0.]
 [0. 2. 0. ... 0. 0. 0.]
 [0. 0. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 3. 0. 0.]
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 0. 0. 1.]]



In [69]:
# Matriz de Laplace calculada
LA = MD - MA
print("Matriz de Laplace calculada a partir das matrizes diagonal de grau e matriz de adjacencia:")
print(LA)
print()

count = 0
for item in LA:
    for item2 in item:
        if(item2 == -1):
            count += 1
print(count)

Matriz de Laplace calculada a partir das matrizes diagonal de grau e matriz de adjacencia:
[[3. 0. 0. ... 0. 0. 0.]
 [0. 2. 0. ... 0. 0. 0.]
 [0. 0. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 3. 0. 0.]
 [0. 0. 0. ... 0. 2. 0.]
 [0. 0. 0. ... 0. 0. 1.]]

27112


## Matriz de penalidades

In [135]:
# escolhendo eleatóriamente os pontos com valores dados
pMatrix = np.zeros([8708, 8708])

b = np.zeros([8708])

# Foram escolhidos 20 valores dados
for i in range(20):
    lineColumn = random.randint(0, 8707)
    pMatrix[lineColumn, lineColumn] = 1e7
    value = random.random() * 10
    b[lineColumn] = value
    print(lineColumn, "= ", value)


7667 =  1.655184601971259
1570 =  4.668483969074958
1037 =  7.275444310204774
5712 =  0.5330411816515945
1363 =  6.847097261975124
6794 =  7.059945413464628
2278 =  6.425136821335814
7118 =  9.350553895307804
6070 =  2.047081240008668
3663 =  5.01589301217415
3100 =  5.360820610802469
818 =  8.431046082495078
1471 =  9.031105821438958
6388 =  7.173454066038969
7748 =  7.467010153490429
2964 =  7.781700111464933
6673 =  7.042613615326361
5695 =  2.6513589998016673
461 =  1.2131526277753457
5161 =  2.6218367746927096


## Comparação do tempo de resolução do sistema

### Decomposição LU

In [149]:
# Decomposição LU
start_time = time.time()

A = (LA + pMatrix)
B = b @ pMatrix

lu, piv = scipy.linalg.lu_factor(A)
x = scipy.linalg.lu_solve((lu, piv), B)
print(x)

print("Solucionar o sistema por meio da decomposição LU tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

[6.50097081 2.98387991 6.15642404 ... 3.81346365 6.26226841 6.79201452]
Solucionar o sistema por meio da decomposição LU tomou:  3.4601645469665527 segundos
É solução do sistema com tolerância 1e-7? 
 True


### Cholesky

In [156]:
# Cholesky
start_time = time.time()

A = (LA + pMatrix)
B = b @ pMatrix

cho, low = scipy.linalg.cho_factor(A)
x = scipy.linalg.cho_solve((cho, low), B)
print(x)

print("Solucionar o sistema por meio do método de Cholesky tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

[6.50097081 2.98387991 6.15642404 ... 3.81346365 6.26226841 6.79201452]
Solucionar o sistema por meio do método de Cholesky tomou:  2.6570887565612793 segundos
É solução do sistema com tolerância 1e-7? 
 True


### Gauss-Jacobi

In [53]:
# Gauss-Jacobi
A = (LA + pMatrix)
B = b @ pMatrix

n = A.shape[0]
def crit_linhas(n):
    condicao = True
    alpha_k = 0
    for k in range(n):
        sum = 0
        for j in range(n):
            if(j != k):
                sum += np.abs(A[k,j])
        alpha_k = sum / np.abs(A[k, k])
        if(alpha_k >= 1): # Se algum dos valores de alpha_k for maior ou igual a 1, o critério das linhas não é satisfeito
            condicao = False
    return condicao

def crit_colunas(n):
    condicao = True
    alpha_k = 0
    for k in range(n):
        sum = 0
        for i in range(n):
            if(i != k):
                sum += np.abs(A[i,k])
        alpha_k = sum / np.abs(A[k, k])
        if(alpha_k >= 1): # Se algum dos valores de alpha_k for maior ou igual a 1, o critério das linhas não é satisfeito
            condicao = False
    return condicao

if (crit_linhas(n) and crit_colunas(n)):
    print("O método de Jacobi PODE ser empregado para resolver o sistema linear.")
else:
    print("O método de Jacobi NÃO PODE ser empregado para resolver o sistema linear pois não passa pelo critério das Linhas e das Colunas.")

O método de Jacobi NÃO PODE ser empregado para resolver o sistema linear pois não passa pelo critério das Linhas e das Colunas


### Gauss-Seidel

In [54]:
# Gauss-Seidel
A = (LA + pMatrix)
B = b @ pMatrix

# O método de Gauss-Seidel converge para a solução Ax = b, independentemente da escolha de x^(0), se satisfazer o critério de Sassenfeld:
# MAX(beta) < 1

n = A.shape[0]
def crit_sassenfeld(n):
    condicao = True
    # Calculando beta_0:
    betas = np.zeros(n)
    num_elements = 0
    sum1 = 0
    for j in range(1,n):
        sum1 += np.abs(A[1,j])
    sum1 = sum1 / np.abs(A[1,1])
    betas[num_elements] = sum1
    num_elements += 1

    # Calculando os demais beta
    sum2 = 0
    for i in range(1,n):
        for j in range(i):
            sum2 += np.abs(A[i,j])*betas[j]
        for j in range(i+1,n):
            sum2 += np.abs(A[i,j])
        sum2 = sum2 / np.abs(A[i,i])
        betas[num_elements] = sum2
        num_elements += 1

    # Checar se o valor máximo dos valores de beta não é maior ou igual a 1
    max = np.max(betas)
    if(max >= 1):
        condicao = False
    return condicao

if (crit_sassenfeld(n)):
    print("O método de Gauss-Seidel PODE ser empregado para resolver o sistema linear.")
else:
    print("O método de Gauss-Seidel NÃO PODE ser empregado para resolver o sistema linear pois não passa pelo critério de Sassenfeld.")

O método de Gauss-Seidel NÃO PODE ser empregado para resolver o sistema linear pois não passa pelo critério de Sassenfeld.


In [ ]:
# # linalg.norm() não conseguia calcular sqrt() pois os valores eram do tipo Float
# def gauss_seidel(A, n, b, x0, tol):
#     L = np.tril(A)
#     R = np.triu(A, 1)
#     C = -scipy.linalg.solve(L, R)
#     g = scipy.linalg.solve(L, b)
#     kmax = 10000
#     k = 0
#     while(normcalc((b - np.dot(A, x0)), x0.size) > tol and k < kmax):
#         k = k + 1
#         x0 = np.dot(C, x0) + g
#     if(k == kmax):
#         print("ERRO: O método não converge!")
#         exit()
#     x = x0
#     return x, k
# def normcalc(x, n):
#     result = 0
#     for i in range(n):
#         result += float(x[0])**2
#     result = np.sqrt(result)
#     return result
#
# def gauss_seidel2(a, x ,b):
#     #Finding length of a(3)
#     n = len(a)
#     # for loop for 3 times as to calculate x, y , z
#     for j in range(0, n):
#         # temp variable d to store b[j]
#         d = b[j]
#
#         # to calculate respective xi, yi, zi
#         for i in range(0, n):
#             if(j != i):
#                 d-=a[j][i] * x[i]
#         # updating the value of our solution
#         x[j] = d / a[j][j]
#     # returning our updated solution
#     return x
#
# start_time = time.time()
#
# # Chute inicial
# x = np.ones([8708])
# #for i in range(0, 4):
#     #x, k = gauss_seidel(A, 8708, B, x, 1e-7)
#     #x = gauss_seidel2(A, x0, B)
# print(x)
#
# print("Solucionar o sistema por meio do método de Cholesky tomou: ",time.time() - start_time,"segundos")
# print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

In [61]:
# No método dos gradientes conjugados a única condição necessária é que a matrix A seja SPD
A = (LA + pMatrix)
B = b @ pMatrix

if (np.all(np.linalg.eigvals(A) > 0)):
    print("O método de Gradientes Conjugados PODE ser empregado para resolver o sistema linear")
else:
    print("O método de Gradientes Conjugados NÃO PODE ser empregado para resolver o sistema linear")

O método de Gradientes Conjugados PODE ser empregado para resolver o sistema linear


In [165]:
# Gradientes Conjugados
A = (LA + pMatrix)
B = b @ pMatrix

start_time = time.time()

x, info = scipy.sparse.linalg.cg(A, B, tol=1e-15, maxiter=1000000)
if(info == 0):
    print("O método converge com a solução: ")
    print(x)
else:
    print("O método não converge!")

print("Solucionar o sistema por meio do método de Gradientes Conjugados tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

O método converge com a solução: 
[6.50097088 2.98387991 6.15642416 ... 3.81346365 6.26226842 6.79201446]
Solucionar o sistema por meio do método de Gradientes Conjugados tomou:  21.03818988800049 segundos
É solução do sistema com tolerância 1e-7? 
 True


## Representação por matriz esparsa

In [141]:
A = (LA + pMatrix)
B = b @ pMatrix

# Matriz esparsa de A
S = scipy.sparse.csc_matrix(A)

start_time = time.time()
print(scipy.sparse.linalg.spsolve(S,B))
print("1: ",time.time() - start_time,"segundos")

start_time = time.time()
print(scipy.linalg.solve(A,B))
print("2: ",time.time() - start_time,"segundos")

[6.50097081 2.98387991 6.15642404 ... 3.81346365 6.26226841 6.79201452]
1:  0.01739192008972168 segundos
[6.50097081 2.98387991 6.15642404 ... 3.81346365 6.26226841 6.79201452]
2:  4.306354761123657 segundos


### Decomposição LU

In [163]:
# Decomposição LU
start_time = time.time()

A = (LA + pMatrix)
B = b @ pMatrix

lu = scipy.sparse.linalg.splu(S)
x = lu.solve(B)
print(x)

print("Solucionar o sistema por meio da decomposição LU tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

[6.50097081 2.98387991 6.15642404 ... 3.81346365 6.26226841 6.79201452]
Solucionar o sistema por meio da decomposição LU tomou:  0.18702292442321777 segundos
É solução do sistema com tolerância 1e-7? 
 True


### Cholesky

In [161]:
# Cholesky
start_time = time.time()

A = (LA + pMatrix)
B = b @ pMatrix

def sparse_cholesky(A): # The input matrix A must be a sparse symmetric positive-definite.
  n = A.shape[0]
  LU = scipy.sparse.linalg.splu(A,diag_pivot_thresh=0) # sparse LU decomposition
  return LU.L.dot(scipy.sparse.diags(LU.U.diagonal()**0.5))

cho = sparse_cholesky(A)
x = cho.solve(B)
print(x)

print("Solucionar o sistema por meio do método de Cholesky tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

AttributeError: solve not found

### Gradientes Conjugados

In [164]:
# Gradientes Conjugados
B = b @ pMatrix

start_time = time.time()

x, info = scipy.sparse.linalg.cg(S, B, tol=1e-15, maxiter=1000000)
if(info == 0):
    print("O método converge com a solução: ")
    print(x)
else:
    print("O método não converge!")

print("Solucionar o sistema por meio do método de Gradientes Conjugados tomou: ",time.time() - start_time,"segundos")
print("É solução do sistema com tolerância 1e-7? \n",np.allclose(A @ x - B, np.zeros((8708,)), rtol=1e-7, atol=1e-7))

O método converge com a solução: 
[6.50097087 2.98387991 6.15642416 ... 3.81346365 6.26226841 6.79201445]
Solucionar o sistema por meio do método de Gradientes Conjugados tomou:  0.11901402473449707 segundos
É solução do sistema com tolerância 1e-7? 
 True
